In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [ ]:
df = pd.read_csv("Weather.csv")

FileNotFoundError: ignored

In [ ]:
df.head()

,Unnamed: 0,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,0,1901,17.99,19.43,23.49,26.41,28.28,28.60,27.49,26.98,26.26,25.08,21.73,18.95
1,1,1902,19.00,20.39,24.10,26.54,28.68,28.44,27.29,27.05,25.95,24.37,21.33,18.78
2,2,1903,18.32,19.79,22.46,26.03,27.93,28.41,28.04,26.63,26.34,24.57,20.96,18.29
3,3,1904,17.77,19.39,22.95,26.73,27.83,27.85,26.84,26.73,25.84,24.36,21.07,18.84
4,4,1905,17.40,17.79,21.78,24.84,28.32,28.69,27.67,27.47,26.29,26.16,22.07,18.71


In [ ]:
df = pd.read_csv("Weather.csv", index_col=0)

In [ ]:
df1 = pd.melt(df, id_vars='YEAR', value_vars=df.columns[1:]) ## This will melt the data
df1.head()

,YEAR,variable,value
0,1901,JAN,17.99
1,1902,JAN,19.00
2,1903,JAN,18.32
3,1904,JAN,17.77
4,1905,JAN,17.40


In [ ]:

df1['Date'] = df1['variable'] + ' ' + df1['YEAR'].astype(str)
df1[df1.columns[df1.columns.get_loc('Date')]] = df1['Date'].apply(lambda x: datetime.strptime(x, '%b %Y'))
df1.head()


,YEAR,variable,value,Date
0,1901,JAN,17.99,1901-01-01
1,1902,JAN,19.00,1902-01-01
2,1903,JAN,18.32,1903-01-01
3,1904,JAN,17.77,1904-01-01
4,1905,JAN,17.40,1905-01-01


In [ ]:
df1.columns=['Year', 'Month', 'Temperature', 'Date']
df1.sort_values(by='Date', inplace=True)
fig = go.Figure(layout = go.Layout(yaxis=dict(range=[0, df1['Temperature'].max()+1])))
fig.add_trace(go.Scatter(x=df1['Date'], y=df1['Temperature']), )
fig.update_layout(title='Temperature Throught Timeline:',xaxis_title='Time', yaxis_title='Temperature in Degrees')
fig.update_layout(xaxis=go.layout.XAxis(rangeselector=dict(buttons=list([dict(label="Whole View", step="all"),dict(count=1,label="One Year View",step="year",stepmode="todate")])), rangeslider=dict(visible=True),type="date"))
fig.show()

In [ ]:
fig = px.box(df1, 'Month', 'Temperature')
fig.update_layout(title='Warmest, Coldest and Median Monthly Temperature.')
fig.show()

In [ ]:
from sklearn.cluster import KMeans
sse = []
target = df1['Temperature'].to_numpy().reshape(-1, 1)
num_clusters = list(range(1, 10))

for k in num_clusters:
    km = KMeans(n_clusters=k, n_init=10)
    km.fit(target)
    sse.append(km.inertia_)

fig = go.Figure(data=[
    go.Scatter(x=num_clusters, y=sse, mode='lines'),
    go.Scatter(x=num_clusters, y=sse, mode='markers')
])

fig.update_layout(
    title="Evaluation on number of clusters:",
    xaxis_title="Number of Clusters:",
    yaxis_title="Sum of Squared Distance",
    showlegend=False
)
fig.show()


In [ ]:
km = KMeans(n_clusters=3, n_init=10)
km.fit(df1['Temperature'].to_numpy().reshape(-1,1))
df1.loc[:,'Temp Labels'] = km.labels_
fig = px.scatter(df1, 'Date', 'Temperature', color='Temp Labels')
fig.update_layout(title = "Temperature clusters.",
                 xaxis_title="Date", yaxis_title="Temperature")
fig.show()

In [ ]:
fig = px.histogram(x=df1['Temperature'], nbins=200, histnorm='density')
fig.update_layout(title='Frequency chart of temperature readings:', xaxis_title='Temperature', yaxis_title='Count')

In [ ]:
df['Yearly Mean'] = df.iloc[:,1:].mean(axis=1) ## Axis 1 for row wise and axis 0 for columns.
fig = go.Figure(data=[
    go.Scatter(name='Yearly Temperatures' , x=df['YEAR'], y=df['Yearly Mean'], mode='lines'),
    go.Scatter(name='Yearly Temperatures' , x=df['YEAR'], y=df['Yearly Mean'], mode='markers')])
fig.update_layout(title='Yearly Mean Temperature :',
                 xaxis_title='Time', yaxis_title='Temperature in Degrees')
fig.show()

In [ ]:
fig = px.line(df1, 'Year', 'Temperature', facet_col='Month', facet_col_wrap=4)
fig.update_layout(title='Monthly temperature throught history:')
fig.show()

In [ ]:
df['Winter'] = df[['DEC', 'JAN', 'FEB']].mean(axis=1)
df['Summer'] = df[['MAR', 'APR', 'MAY']].mean(axis=1)
df['Monsoon'] = df[['JUN', 'JUL', 'AUG', 'SEP']].mean(axis=1)
df['Autumn'] = df[['OCT', 'NOV']].mean(axis=1)
seasonal_df = df[['YEAR', 'Winter', 'Summer', 'Monsoon', 'Autumn']]
seasonal_df = pd.melt(seasonal_df, id_vars='YEAR', value_vars=seasonal_df.columns[1:])
seasonal_df.columns=['Year', 'Season', 'Temperature']

fig = px.scatter(seasonal_df, 'Year', 'Temperature', facet_col='Season', facet_col_wrap=2, trendline='ols')
fig.update_layout(title='Seasonal mean temperatures through years:')
fig.show()

NameError: ignored

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

df2 = df1[['Year', 'Month', 'Temperature']].copy()
df2 = pd.get_dummies(df2)
y = df2[['Temperature']]
x = df2.drop(columns='Temperature')

dtr = DecisionTreeRegressor()
train_x, test_x, train_y, test_y = train_test_split(x,y,test_size=0.3)
dtr.fit(train_x, train_y)
pred = dtr.predict(test_x)
r2_score(test_y, pred)

0.9702446030841242